# B) PCI Controls and Audit checks

PCI basic checks for SafePay project

- Confirm PAN is not stored in repo artifacts.
- Confirm tokenization outputs exist and tokens are unique.
- Confirm baseline policy artifact exists.
- Produce basic evidences for audit consumption.

In [ ]:
from pathlib import Path
import re, csv, json
import datetime, hashlib
import pandas as pd

BASE = Path('..').resolve()
output_path = Path("../outputs").resolve()
token_path = Path("../../A_Payment_Dataflow/outputs/tokenized_payments.csv").resolve()
pci_baseline = Path("../../../02_Framework/pci_baseline.json").resolve()

print(output_path)
print(token_path)
print(pci_baseline)

C:\Project\SafePay\02_Framework\pci_baseline.json
C:\Project\SafePay\01_Case\A_Payment_Dataflow\outputs\tokenized_payments.csv
C:\Project\SafePay\01_Case\B_PCI_Controls_and_Audit\outputs


In [18]:
checks = []  

# 1) validate
if token_path.exists():
    checks.append(("CHK-001","tokenized_file_present","PASS", str(token_path)))
else:
    checks.append(("CHK-001","tokenized_file_present","FAIL","tokenized file not found"))

if baseline_path.exists():
    checks.append(("CHK-002","baseline_present","PASS", str(baseline_path)))
else:
    checks.append(("CHK-002","baseline_present","FAIL","pci_baseline.json not found"))

# 2) validate tokens
pan_pattern = re.compile(r"\b\d{13,19}\b")  # PAN pattern
if token_path.exists():
    try:
        df = pd.read_csv(token_path, dtype=str, keep_default_na=False)

        # a) PAN leak check
        pan_cols = [col for col in df.columns if df[col].astype(str).str.contains(pan_pattern).any()]
        if pan_cols:
            checks.append(("CHK-003","pan_leak_check","FAIL", f"PAN-like sequences found in columns: {pan_cols}"))
        else:
            checks.append(("CHK-003","pan_leak_check","PASS","No PAN-like sequences found"))

        # b) Token uniqueness
        if 'token' in df.columns:
            total = len(df)
            unique = df['token'].nunique()
            if total == unique:
                checks.append(("CHK-004","token_uniqueness","PASS", f"total={total}, unique={unique}"))
            else:
                checks.append(("CHK-004","token_uniqueness","FAIL", f"total={total}, unique={unique}"))
        else:
            checks.append(("CHK-004","token_uniqueness","FAIL","'token' column not found"))

        # c) Status counts
        if 'status' in df.columns:
            counts = df['status'].value_counts().to_dict()
            checks.append(("CHK-005","status_counts","INFO", json.dumps(counts)))
        else:
            checks.append(("CHK-005","status_counts","INFO","status column missing"))
    except Exception as e:
        checks.append(("CHK-ERROR","token_file_parse_error","FAIL", str(e)))

print ("Validation done")

Validation done


In [21]:
# 3) Save results
evidence_csv = output_path / "pci_evidence.csv"
with open(evidence_csv, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(['check_id','check_name','result','details'])
    for r in checks:
        w.writerow(r)

detail_json = output_path / "pci_checks_detail.json"
detail = {"run_ts": datetime.datetime.now().isoformat()+"Z", "checks": []}
for cid, name, res, details in checks:
    detail["checks"].append({"id": cid, "name": name, "result": res, "details": details})
detail_json.write_text(json.dumps(detail, indent=2))

# 4) Show results here
print("Simple PCI checks completed. Summary:")
for cid, name, res, details in checks:
    print(f"- {cid} | {name} | {res} | {details}")

pd.read_csv(evidence_csv)

Simple PCI checks completed. Summary:
- CHK-001 | tokenized_file_present | PASS | C:\Project\SafePay\01_Case\A_Payment_Dataflow\outputs\tokenized_payments.csv
- CHK-002 | baseline_present | PASS | C:\Project\SafePay\01_Case\B_PCI_Controls_and_Audit
- CHK-003 | pan_leak_check | PASS | No PAN-like sequences found
- CHK-004 | token_uniqueness | PASS | total=1200, unique=1200
- CHK-005 | status_counts | INFO | {"failed": 423, "success": 398, "declined": 379}


,check_id,check_name,result,details
0,CHK-001,tokenized_file_present,PASS,C:\Project\SafePay\01_Case\A_Payment_Dataflow\...
1,CHK-002,baseline_present,PASS,C:\Project\SafePay\01_Case\B_PCI_Controls_and_...
2,CHK-003,pan_leak_check,PASS,No PAN-like sequences found
3,CHK-004,token_uniqueness,PASS,"total=1200, unique=1200"
4,CHK-005,status_counts,INFO,"{""failed"": 423, ""success"": 398, ""declined"": 379}"
